# Ανάκτηση Πληροφορίας 
Δροσερή Λουκά 19390121 - <br>
Ευθύμης Μάντης 19390129

Σκοπός της Εργασίας:
Η παρούσα εργασία αποσκοπεί στον σχεδιασμό και την υλοποίηση ενός συστήματος ανάκτησης εγγράφων,
το οποίο θα μπορεί να ευρετηριάζει και να αναζητά αποτελεσματικά σε μια συλλογή κειμένων. Στο πλαίσιο της εργασίας, οι στόχοι περιλαμβάνουν:

1. Τη σχεδίαση και υλοποίηση αλγορίθμων ανάκτησης εγγράφων, που θα αξιοποιούν μοντέρνες μεθόδους, όπως Boolean retrieval, Vector Space Model και Probabilistic retrieval models.
2. Την αξιολόγηση της απόδοσης των αλγορίθμων ανάκτησης χρησιμοποιώντας κοινές μετρήσεις αξιολόγησης, όπως την ακρίβεια (precision), την ανάκληση (recall) και τη βαθμολογία F1.
3. Την ανάπτυξη ενός φιλικού προς το χρήστη περιβάλλοντος, μέσω του οποίου οι χρήστες θα μπορούν να εισάγουν ερωτήματα και να ανακτούν σχετικά έγγραφα.
4. Την απόκτηση πρακτικής εμπειρίας σε τεχνικές ανάκτησης πληροφοριών.

# Bήμα 1

# Εισαγωγή Βιβλιοθηκών && Λήψη Πόρων του NLTK
1. requests:<br> Χρησιμοποιείται για την αποστολή HTTP αιτημάτων και την ανάκτηση περιεχομένου από ιστοσελίδες.<br>
2. BeautifulSoup:<br> Χρησιμοποιείται για την ανάλυση HTML και XML, ώστε να εξάγονται δεδομένα από ιστοσελίδες.<br>
3. pandas:<br> Βιβλιοθήκη για τη διαχείριση και ανάλυση δεδομένων σε μορφή πινάκων (DataFrames).<br>
4. re: <br>Παρέχει λειτουργίες για την εργασία με κανονικές εκφράσεις.<br>
5. nltk: <br>Χρησιμοποιείται για την επεξεργασία φυσικής γλώσσας (tokenization, stopwords, lemmatization).<br>
6. TfidfVectorizer:<br> Εργαλείο της scikit-learn για τον υπολογισμό της TF-IDF αναπαράστασης κειμένων.<br>
7. BM25Okapi: <br>Υλοποίηση του αλγορίθμου BM25 για την αξιολόγηση σχετικότητας κειμένων.<br>
8. cosine_similarity:<br> Υπολογίζει την συνημιτονοειδή ομοιότητα μεταξύ διανυσμάτων, χρήσιμη για την σύγκριση κειμένων.<br>
<br>
_______________________________________________________________________________________________________________________________________________________________________________________________________
<br>

1. nltk.download: <br>Λήψη των απαραίτητων πόρων του NLTK που απαιτούνται για την εκτέλεση συγκεκριμένων λειτουργιών.<br>
2. punkt: <br>Περιλαμβάνει δεδομένα για τον διαχωρισμό προτάσεων και λέξεων (tokenization).<br>
3. stopwords: <br>Λίστα με τις κοινές λέξεις που συχνά παραλείπονται (π.χ., "the", "is").<br>
4. wordnet: <br>Λεξικό που χρησιμοποιείται για τη λεματοποιήση (lemmatization).<br>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Δροσερή\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Δροσερή\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Δροσερή\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Δροσερή\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Συλλογή Δεδομένων && Αποθήκευση Παραγράφων
Δημιουργούμε μια λίστα που περιέχει τα URLs των άρθρων που θέλουμε να συλλέξουμε από τη Wikipedia.<br>
Αυτή η λίστα μπορεί να επεκταθεί με περισσότερους συνδέσμους για την εξαγωγή περιεχομένου από πολλαπλές πηγές.<br>
Αρκεί μόνο να κάνουμε copy - paste την wikipedia σελίδα που θέλουμε να εξετάσουμε , μέσα στο πεδίο url.
<br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>

**1. all_paragraphs:**  Μια κενή λίστα όπου θα αποθηκεύονται όλες οι παράγραφοι που εξάγονται από τα άρθρα της Wikipedia.<br>
Για παράδειγμα, αν ένα άρθρο έχει 10 παραγράφους, αυτές θα προστεθούν στη λίστα αυτή με τη σειρά που εμφανίζονται στο άρθρο.

**2. cleaned_paragraphs:** Μια δεύτερη κενή λίστα που θα περιέχει τις παραγράφους αφού έχουν υποστεί προ-επεξεργασία.<br>
Η σείρα με την οποιά θα εισαχθούν στη λίστα είναι ίδια με την all_paragraphs<br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>
Προ-επεξεργασία περιλαμβάνει:
<br>
Αφαίρεση ειδικών χαρακτήρων.<br>
Αφαίρεση κενών γραμμών.<br>
Επεξεργασία για βελτιωμένη ανάλυση και αναζήτηση.<br>

In [3]:

# Βήμα 1. Συλλογή δεδομένων:
# Λίστα με τα URLs των άρθρων που θέλεις να συλλέξεις
urls = [
" https://en.wikipedia.org/wiki/Buprenorphine",
 "https://en.wikipedia.org/wiki/Paracetamol"
]


all_paragraphs = []  # Λίστα για να αποθηκεύσουμε όλες τις παραγράφους
cleaned_paragraphs = []  # Λίστα για να αποθηκεύσουμε όλες τις παραγράφους μετά την προ-επεξεργασία

# Συλλογή και Επεξεργασία Παραγράφων από URLs
**1. Επανάληψη μέσω των URLs**<br>
Ο κώδικας χρησιμοποιεί ένα βρόχο for για να επεξεργαστεί κάθε URL που βρίσκεται στη λίστα urls. Για κάθε URL, πραγματοποιείται ένα HTTP αίτημα μέσω της συνάρτησης requests.get(url).<br><br>
**2. Έλεγχος της Απόκρισης**<br>
Με την εντολή if response.status_code == 200, ο κώδικας ελέγχει αν το αίτημα ήταν επιτυχές. Ο κωδικός 200 δείχνει ότι το αίτημα HTTP ολοκληρώθηκε χωρίς προβλήματα.<br>
Εάν το αίτημα αποτύχει (π.χ., ο ιστότοπος δεν είναι διαθέσιμος), εκτυπώνεται ένα μήνυμα σφάλματος που περιλαμβάνει το URL και τον κωδικό σφάλματος.<br><br>
**3. Επεξεργασία HTML**<br>
Η βιβλιοθήκη BeautifulSoup χρησιμοποιείται για να αναλύσει το HTML περιεχόμενο της σελίδας. Το αντικείμενο soup περιέχει τη δομή της σελίδας με τρόπο που επιτρέπει εύκολη αναζήτηση και εξαγωγή δεδομένων.<br><br>
**4. Εξαγωγή Παραγράφων**<br>
Με την εντολή soup.find_all('p'), ο κώδικας βρίσκει όλα τα στοιχεία < p > της HTML, τα οποία περιέχουν παραγράφους κειμένου.<br>
Για κάθε παράγραφο, το .text.strip() αφαιρεί τυχόν περιττά κενά ή χαρακτήρες, ώστε το κείμενο να είναι καθαρό.<br>
Η συνθήκη if p.text.strip() != "" απορρίπτει τις κενές παραγράφους.<br><br>
**5.Αποθήκευση Παραγράφων**<br>
Η λίστα all_paragraphs χρησιμοποιείται για να αποθηκεύσει όλες τις παραγράφους που εξάγονται. Οι παραγράφοι κάθε σελίδας προστίθενται σε αυτήν με την εντολή all_paragraphs.extend().<br><br>
**6.Διαχείριση Σφαλμάτων**<br>
Αν η απόκριση HTTP αποτύχει (π.χ., ο ιστότοπος δεν είναι διαθέσιμος), το πρόγραμμα εμφανίζει ένα μήνυμα που ενημερώνει τον χρήστη για το URL και τον λόγο αποτυχίας. Αυτό βοηθάει στον εντοπισμό προβλημάτων κατά την εκτέλεση.<br>


In [4]:
# Επανάληψη για κάθε URL στη λίστα
for url in urls:
    response = requests.get(url)

    # Έλεγχος για επιτυχία στην απόκριση
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # Εξαγωγή παραγράφων από τη σελίδα
        global paragraphs
        paragraphs = soup.find_all('p')
        parsed_paragraphs = [p.text.strip() for p in paragraphs if p.text.strip() != ""]


        # Προσθήκη των παραγράφων στη λίστα all_paragraphs
        all_paragraphs.extend(parsed_paragraphs)
    else:
        print(f"Αποτυχία φόρτωσης σελίδας: {url} (Κωδικός Σφάλματος: {response.status_code})")


# Αποθήκευση Δεδομένων σε Αρχείο CSV
**1. Δημιουργία λίστας δεδομένων (data)**<br>
Με την εντολή data = [{'Paragraph': paragraph} for paragraph in all_paragraphs], δημιουργούμε μια λίστα data.<br>
Κάθε στοιχείο της λίστας είναι ένα λεξικό (dict), όπου το κλειδί είναι Paragraph και η τιμή είναι η κάθε παράγραφος που εξήχθη από τις σελίδες.<br>
Το αποτέλεσμα είναι μια λίστα που περιέχει όλα τα δεδομένα, έτοιμη για να χρησιμοποιηθεί σε μεταγενέστερα βήματα.<br><br>
**2. Δημιουργία DataFrame (pd.DataFrame)**<br>
Το df = pd.DataFrame(data) μετατρέπει τη λίστα data σε ένα DataFrame, το οποίο είναι ένα δομημένο αντικείμενο δεδομένων της βιβλιοθήκης pandas.<br>
Κάθε στοιχείο της λίστας μετατρέπεται σε row του πίνακα, με την στήλη να έχει τίτλο Paragraph που περιέχει τις παραγράφους.<br><br>
**3. Αποθήκευση CSV**<br>
Με την εντολή df.to_csv('wikipedia_multiple_articles.csv', index=False, encoding='utf-8'), το DataFrame αποθηκεύεται σε αρχείο CSV.<br>
Το index=False διασφαλίζει ότι η σειρά των γραμμών δε θα αποθηκευτεί σαν ξεχωριστή στήλη στο CSV.<br>
Το encoding='utf-8' εξασφαλίζει ότι τα δεδομένα αποθηκεύονται σε UTF-8, το οποίο υποστηρίζει ελληνικούς χαρακτήρες και άλλες γλώσσες.<br>

In [5]:

# Αποθήκευση των δεδομένων σε CSV
data = [{'Paragraph': paragraph} for paragraph in all_paragraphs]
df = pd.DataFrame(data)
# Αποθήκευση στο αρχείο CSV
df.to_csv('wikipedia_multiple_articles.csv', index=False, encoding='utf-8')
print("Τα δεδομένα αποθηκεύτηκαν στο 'wikipedia_multiple_articles.csv'")

Τα δεδομένα αποθηκεύτηκαν στο 'wikipedia_multiple_articles.csv'


# Βήμα 2
	

# Δημιουργία Stop Words και Lemmatizer για την Επεξεργασία Κειμένου
1. stop_words<br>
Σκοπός: Δημιουργείται μια λίστα από τις κοινές λέξεις της Αγγλικής γλώσσας (stop words), που συνήθως δεν προσφέρουν χρήσιμη πληροφορία για τις αναζητήσεις ή τη στατιστική ανάλυση κειμένων.<br>
Λειτουργία: Οι λέξεις αυτές, όπως "the", "is", "and", "in", "to", κ.λπ., θα αφαιρεθούν από το κείμενο κατά τη διάρκεια της επεξεργασίας ώστε να επικεντρωθούμε μόνο σε πιο σημαντικές λέξεις.<br>
Χρήση: Οι λέξεις αυτές δε συμπεριλαμβάνονται στη δημιουργία του TF-IDF ή άλλων μετρικών ανάλυσης κειμένου.<br><br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
2. lemmatizer<br>
Σκοπός: Δημιουργείται ο WordNetLemmatizer, που είναι υπεύθυνος για την ενοποίηση των διαφορετικών μορφών μιας λέξης σε μία στατική μορφή.<br>
Λειτουργία: Π.χ., το "running", "runs", και "ran" θα μετατραπούν σε "run", ή "better" σε "good". Αυτό είναι χρήσιμο για την ομαδοποίηση λέξεων και την εξάλειψη της συχνότητας που εμφανίζεται κάθε μορφή μιας λέξης.<br>
Χρήση: Ο lemmatizer χρησιμοποιείται κυρίως σε βήματα επεξεργασίας κειμένου για την ενοποίηση των διαφορετικών γραμμών ή λέξεων, ώστε να γίνεται πιο αποτελεσματική η αναζήτηση ή ανάλυση περιεχομένου.<br>

In [6]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Προεπεξεργασία Κειμένου για Ανάλυση
**1. Αφαίρεση Ειδικών Χαρακτήρων και Αριθμών:**<br>
Το πρώτο βήμα περιλαμβάνει την αφαίρεση όλων των ειδικών χαρακτήρων και των αριθμών από το κείμενο, αφήνοντας μόνο γράμματα και κενά. Χρησιμοποιώντας την μέθοδο re.sub(), καθαρίζεται το κείμενο από μη απαραίτητα σύμβολα.<br><BR>

**2. Tokenization (Διαχωρισμός Κειμένου σε Λέξεις):**<br>
Το κείμενο διαχωρίζεται σε λέξεις μέσω της μεθόδου word_tokenize() της βιβλιοθήκης NLTK. Αυτή η διαδικασία δημιουργεί μια λίστα με τις λέξεις που βρίσκονται στο κείμενο.<br><br>

**3. Αφαίρεση Stop Words:**<br>
Οι κοινές λέξεις της Αγγλικής (stop words) που δεν συνεισφέρουν στην ανάλυση, όπως το "και", "το", "αυτό", αφαιρούνται από το κείμενο. Το tokens περνάει από μια λίστα stop words και μόνο οι σημαντικές λέξεις διατηρούνται.<br><br>

**4. Lemmatization (Μείωση σε Βασική Μορφή Λέξης):**<br>
Η διαδικασία της lemmatization μειώνει τις λέξεις στην κύρια μορφή τους. Έτσι, διαφορετικές μορφές της ίδιας λέξης, όπως "running", "runs", "ran", γίνονται μία ενιαία μορφή (π.χ., "run").<br><br>

**Τελικό Αποτέλεσμα:** <br>
Το τελικό αποτέλεσμα αυτής της διαδικασίας είναι μια λίστα από λεmmatized λέξεις, έτοιμες για περαιτέρω ανάλυση. Αυτή η μορφή του κειμένου είναι ιδανική για εξαγωγή χαρακτηριστικών, αναζήτηση πληροφοριών ή ακόμα και εκπαίδευση μοντέλων.<br> <br>



In [7]:

# Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):
def preprocess_text(text):
    # Το αποτέλεσμα του βήματος περνάει στο επόμενο
    # 1. Αφαίρεση ειδικών χαρακτήρων και αριθμών
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 2. Tokenization: Διαχωρισμός κειμένου σε λέξεις
    tokens = word_tokenize(text.lower())  # Όλα τα γράμματα σε πεζά
    
    # 3. Αφαίρεση stop words
    tokens = [word for word in tokens if word not in stop_words]
    
    # 4. Lemmatization: Μείωση σε βασική μορφή λέξης
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    #Επιστρέφω την τελική λίστα
    return lemmatized_tokens


# Εκτύπωση && Καθαρισμός Παραγράφων
Κάθε παράγραφος περνά μέσα από τη συνάρτηση preprocess_text, η οποία εκτελεί τέσσερα βήματα επεξεργασίας κειμένου.
 Συνοπτικά τα βήματα είναι (βλέπε προηγούμενο block κώδικα για μεγαλύτερη ανάλυση)<br>
**1. Αφαίρεση ειδικών χαρακτήρων και αριθμών:** Το re.sub(r'[^a-zA-Z\s]', '', text) αφαιρεί κάθε χαρακτήρα που δεν είναι γράμμα ή κενό.<br><br>
**2. Tokenization:** Το word_tokenize(text.lower()) διαχωρίζει το κείμενο σε λέξεις και τις μετατρέπει σε πεζά.<br><br>
**3. Αφαίρεση stop words:** Ο προεπιλεγμένος κατάλογος stop_words χρησιμοποιείται για την αφαίρεση κοινών λέξεων που δεν προσφέρουν πληροφορία.<br><br>
**4. Lemmatization:** Το WordNetLemmatizer() μειώνει τις λέξεις σε μια βασική τους μορφή (λ.χ., από το "running" γίνεται "run").<br><br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
Σχετικά με τη λειτουργικότητα της συνάρτησης:<br>
1. Κάθε παράγραφος εκτυπώνεται αρχικά.
2. Στη συνέχεια, εφαρμόζεται η συνάρτηση preprocess_text και το αποτέλεσμα εκτυπώνεται για να δει κανείς τη μετατροπή.
3. Το καθαρισμένο κείμενο αποθηκεύεται στη λίστα cleaned_paragraphs.

In [8]:
for paragraph in all_paragraphs:
    # Εκτύπωση του αρχικού κειμένου
    print("Πριν την προ-επεξεργασία:")
    print(paragraph)
    print("\n" + "-"*50 + "\n")

    # Εφαρμογή προ-επεξεργασίας στην παράγραφο
    cleaned_paragraph = preprocess_text(paragraph)

    # Εκτύπωση του καθαρισμένου κειμένου
    print("Μετά την προεπεξεργασία:")
    print(cleaned_paragraph)
    print("\n" + "="*50 + "\n")
    cleaned_paragraphs.append(cleaned_paragraph)

Πριν την προ-επεξεργασία:
Buprenorphine, sold under the brand name Subutex among others, is an opioid used to treat opioid use disorder, acute pain, and chronic pain.[18] It can be used under the tongue (sublingual), in the cheek (buccal), by injection (intravenous and subcutaneous), as a skin patch (transdermal), or as an implant.[18][19] For opioid use disorder, the patient must have moderate opioid withdrawal symptoms before buprenorphine can be administered under direct observation of a health-care provider.[18]

--------------------------------------------------

Μετά την προεπεξεργασία:
['buprenorphine', 'sold', 'brand', 'name', 'subutex', 'among', 'others', 'opioid', 'used', 'treat', 'opioid', 'use', 'disorder', 'acute', 'pain', 'chronic', 'pain', 'used', 'tongue', 'sublingual', 'cheek', 'buccal', 'injection', 'intravenous', 'subcutaneous', 'skin', 'patch', 'transdermal', 'implant', 'opioid', 'use', 'disorder', 'patient', 'must', 'moderate', 'opioid', 'withdrawal', 'symptom', 'b

# Αποθήκευση Καθαρισμένων Δεδομένων σε CSV
Αυτός ο κώδικας αποθηκεύει τα καθαρισμένα δεδομένα σε ένα αρχείο CSV.<br>
Τα "καθαρισμένα" δεδομένα  (cleaned_data) βρίσκονται στη λίστα cleaned_paragraphs, όπου κάθε στοιχείο είναι μια παράγραφος που έχει τεθεί σε επεξεργασία μέσω της συνάρτησης preprocess_text.<br> 

**1. Δημιουργία cleaned_data:** <br> Μετατρέπει κάθε καθαρισμένη παράγραφο σε ένα αντικείμενο dictionary με το κλειδί Paragraph.<br>
**2. Δημιουργία DataFrame:**<br> Ο πίνακας pd.DataFrame(cleaned_data) δημιουργεί ένα DataFrame από αυτά τα dictionary.<br>
**3. Αποθήκευση στο CSV:** <br> Το df.to_csv('wikipedia_multiple_articles_cleaned_version.csv', index=False, encoding='utf-8') αποθηκεύει το DataFrame σε αρχείο CSV με το όνομα wikipedia_multiple_articles_cleaned_version.csv, χωρίς τον δείκτη και με κωδικοποίηση UTF-8.<br>
**4. Εκτύπωση επιτυχίας:**<br> Μετά την αποθήκευση, εμφανίζεται ένα μήνυμα ότι τα δεδομένα αποθηκεύτηκαν επιτυχώς.<br>

In [9]:
# Αποθήκευση των δεδομένων σε CSV
cleaned_data = [{'Paragraph': paragraph} for paragraph in cleaned_paragraphs]
df = pd.DataFrame(cleaned_data)
# Αποθήκευση στο αρχείο CSV
df.to_csv('wikipedia_multiple_articles_cleaned_version.csv', index=False, encoding='utf-8')
print("Τα δεδομένα προ-επεξεργασίας αποθηκεύτηκαν στο 'wikipedia_multiple_articles_cleaned_version.csv'")

Τα δεδομένα προ-επεξεργασίας αποθηκεύτηκαν στο 'wikipedia_multiple_articles_cleaned_version.csv'


# Βήμα 3.

# Δημιουργία Ευρετηρίου (Inverted Index)
Ο κώδικας που ακολουθεί δημιουργεί το ευρετήριο (inverted index) από τις παραγράφους που έχουν ήδη περάσει από την προ-επεξεργασία με τη συνάρτηση preprocess_text. Το ευρετήριο είναι ένα λεξικό όπου το κάθε κλειδί είναι μια λέξη, και οι τιμές του είναι οι δείκτες των παραγράφων (ποιοι παραγράφοι είναι πχ 1, 2, 3, 10) στις οποίες εμφανίζεται αυτή η λέξη.<br>


**1. Δημιουργία inverted_index:**<br>

Αρχικά δημιουργείται ένα άδειο λεξικό inverted_index που θα περιέχει τις λέξεις ως κλειδιά.<br><br>
**2. Επεξεργασία κάθε παραγράφου:**<br>

Κάθε παράγραφος περνάει από την προ-επεξεργασία (preprocess_text), όπου οι λέξεις καθαρίζονται, μειώνονται στη βάση τους (lemmatized), και αφαιρούνται οι stop words.<br><br>
**3. Ενημέρωση του ευρετηρίου:**<br>

Για κάθε λέξη που συναντάται σε μια παράγραφο, αν αυτή δεν υπάρχει ακόμα στο ευρετήριο, προστίθεται με μια κενή λίστα.<br>
Αν η λέξη υπάρχει στο ευρετήριο, τότε προστίθεται ο δείκτης (index) της τρέχουσας παραγράφου, η οποία είναι idx.<br><br>
**4. Επιστροφή του ευρετηρίου:**<br>

Τελικό αποτέλεσμα είναι το λεξικό όπου κάθε λέξη συσχετίζεται με τις παραγράφους που την περιέχουν.

In [10]:

# Βήμα 3. Ευρετήριο (Indexing):
def create_inverted_index(paragraphs):
    #δημιουργία άδειου λεξικού
    inverted_index = {}

    for idx, paragraph in enumerate(paragraphs):

        # Προεπεξεργασία κάθε παραγράφου
        lemmatized_tokens = preprocess_text(paragraph)

        for word in lemmatized_tokens:
            if word not in inverted_index:
                inverted_index[word] = []
            inverted_index[word].append(idx)  # Προσθήκη του index της παραγράφου στη λέξη
    #Επιστρέφω τo τελικό λεξικό , ευρετήριο
    return inverted_index

# Εμφάνιση Αντεστραμμένου Ευρετηρίου
Ο κώδικας παρακάτω καλεί τη συνάρτηση create_inverted_index που περιέχει τη διαδικασία για τη δημιουργία του αντεστραμμένου ευρετηρίου. Το ευρετήριο αυτό αποθηκεύει τις λέξεις ως κλειδιά και τις σχετίζει με τους δείκτες των παραγράφων στις οποίες εμφανίζονται.<br>

**1. create_inverted_index(all_paragraphs):<br>**

Εκτελεί την διαδικασία της δημιουργίας του αντεστραμμένου ευρετηρίου, όπως περιγράφηκε προηγουμένως. Καταγράφει τη σχέση κάθε λέξης με τις παραγράφους που την περιέχουν.<br> <br>
**2. print("Αντεστραμμένο Ευρετήριο:"):<br>**

Εκτυπώνει ένα ξεκάθαρο μήνυμα ώστε να δίνεται έμφαση ότι το επόμενο μέρος του κώδικα εμφανίζει το αντεστραμμένο ευρετήριο.<br><br>
**3. for word, paragraphs in inverted_index.items():<br>**

Περιηγείται σε κάθε ζεύγος (λέξη, παράγραφοι) από το αντεστραμμένο ευρετήριο.<br><br>
**4. print(f"Λέξη: {word} | παράγραφοι: {', '.join(map(str, paragraphs))}"):<br>**

Εκτυπώνει κάθε λέξη και τις αντίστοιχες παράγραφους όπου η λέξη αυτή εμφανίζεται. Οι παράγραφοι εκτυπώνονται ως σειρές αριθμών, που αντιστοιχούν στους δείκτες τους στο αρχικό σύνολο all_paragraphs.<br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>
Περισσότερη ανάλυση εντολής:

**print(f"Λέξη: {word} | παράγραφοι : {', '.join(map(str, paragraphs))}"):**
<br>
1. Εκτυπώνει τη λέξη (word), ακολουθούμενη από τις παραγράφους όπου βρίσκεται αυτή η λέξη.<br>
', '.join(map(str, paragraphs)) μετατρέπει τους αριθμούς των παραγράφων σε strings(map) και τα ενώνει (join) με ",", π.χ. 0, 3, 5, δείχνοντας σε ποια σημεία του κειμένου η λέξη εμφανίζεται.

In [11]:
inverted_index = create_inverted_index(all_paragraphs)
print("Αντεστραμμένο Ευρετήριο:")
for word, paragraphs in inverted_index.items():
    print(f"Λέξη: {word} | παράγραφοι : {', '.join(map(str, paragraphs))}")


Αντεστραμμένο Ευρετήριο:
Λέξη: buprenorphine | παράγραφοι : 0, 0, 1, 1, 1, 1, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 7, 7, 7, 7, 7, 7, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 13, 13, 15, 16, 17, 17, 18, 19, 20, 21, 22, 23, 24, 25, 25, 25, 26, 26, 26, 26, 26, 26, 27, 27, 28, 29, 30, 30, 30, 30, 31, 31, 31, 31, 33, 33, 33, 34, 35, 35, 37, 37, 38, 39, 39, 40, 41, 41, 42, 42, 43
Λέξη: sold | παράγραφοι : 0, 45, 98, 104
Λέξη: brand | παράγραφοι : 0, 37, 45, 48, 104, 109
Λέξη: name | παράγραφοι : 0, 37, 37, 40, 45, 48, 94, 97, 103, 103, 103, 103, 103, 103, 103, 104, 105, 109
Λέξη: subutex | παράγραφοι : 0, 35, 37
Λέξη: among | παράγραφοι : 0, 2, 35, 48
Λέξη: others | παράγραφοι : 0, 48
Λέξη: opioid | παράγραφοι : 0, 0, 0, 0, 1, 1, 1, 2, 2, 3, 4, 4, 5, 7, 7, 7, 7, 7, 7, 8, 13, 13, 13, 16, 18, 19, 22, 30, 30, 31, 33, 33, 35, 37, 38, 38, 39, 41, 67
Λέξη: used | παράγραφοι : 0, 0, 3, 3, 4, 6, 7, 9, 9, 13, 37, 37, 37, 42, 42, 45, 47, 47, 48, 49, 49, 51, 52, 56, 65, 66, 69, 69, 76, 93, 103, 103, 103, 105


# Βήμα 4. 



**Χρήση:** <br>
Ο χρήστης εισάγει το choice του (1, 2, 3 ή 4).<br>
Αν επιλέξει 4, τότε το πρόγραμμα τερματίζει.<br>
Αν επιλέξει 1, 2, ή 3, το πρόγραμμα περνάει στη δεύτερη φάση, όπου ζητάει το ερώτημα του χρήστη.<br>
Αυτό το κομμάτι κώδικα είναι μια ολοκληρωμένη προσέγγιση για τη δημιουργία μιας μηχανής αναζήτησης που υποστηρίζει διαφορετικούς αλγορίθμους, όπως TF-IDF, Boolean Retrieval, και Okapi BM25. Ο χρήστης μπορεί να επιλέξει την <br>προτιμώμενη μέθοδο αναζήτησης, να εισαγάγει ένα ερώτημα και να λάβει αποτελέσματα από τις παραγράφους που αναζητούνται.<br><br>

**1. Ανάλυση Βημάτων και Λειτουργίας**
1. Μενού Επιλογών<br>
* Το πρόγραμμα ξεκινάει εμφανίζοντας τις διαθέσιμες επιλογές για τους αλγορίθμους:<br>
* Ο χρήστης εισάγει την επιλογή του μέσω της μεταβλητής choice.<br>
1. TF-IDF<br>
2. Boolean Retrieval<br>
3. Okapi BM25<br>
4. Exit<br>

**2. Αναζήτηση με TF-IDF<br>**
Η μέθοδος compute_tfidf δημιουργεί τον TF-IDF πίνακα και τους όρους.<br>
Το ερώτημα προεπεξεργάζεται, και στη συνέχεια η συνάρτηση search_tfidf υπολογίζει τις ομοιότητες μεταξύ του ερωτήματος και κάθε παραγράφου.<br>
Τα αποτελέσματα κατατάσσονται με βάση τη σχετικότητα (descending order), και οι 5 πιο σχετικές παράγραφοι εκτυπώνονται μαζί με τα σκορ τους.<br><br>

**3. Αναζήτηση με Boolean Retrieval<br>**
Το ερώτημα χωρίζεται σε λέξεις και τελεστές (e.g., AND, OR, NOT).<br>
Οι όροι επεξεργάζονται με τη μέθοδο preprocess_text, ενώ οι τελεστές αναγνωρίζονται και διαχειρίζονται.<br>
Το ευρετήριο ανατρέχεται για την εύρεση των παραγράφων που περιέχουν τους όρους του ερωτήματος.<br><br>

**Αν υπάρχουν τελεστές:<br>**
AND: Υπολογίζονται οι παραγράφοι που περιέχουν όλους τους όρους.<br>
NOT: Αποκλείονται παραγράφοι που περιέχουν τον όρο μετά τον τελεστή.<br>
OR: Υπολογίζονται οι παράγραφοι που περιέχουν τουλάχιστον έναν από τους όρους.<br>

**Χωρίς τελεστές:**<br> Όλες οι παραγράφοι που περιέχουν τουλάχιστον μία λέξη εμφανίζονται.<br>
Τα αποτελέσματα εκτυπώνονται με τη σειρά τους.<br><br>

**4. Αναζήτηση με Okapi BM25<br>**
Υπολογίζονται οι βαθμολογίες BM25 για κάθε παράγραφο με τη βοήθεια της συνάρτησης compute_bm25.<br>
Η συνάρτηση search_bm25 αναλύει το ερώτημα και κατατάσσει τις παραγράφους βάσει της σχετικότητάς τους.<br>
Τα αποτελέσματα εμφανίζονται με τη σειρά σχετικότητας.<br><br>

**5. Έξοδος<br>**
Εάν ο χρήστης εισάγει 4, το πρόγραμμα σταματά την εκτέλεση.<br><br>

____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>

# Ανάλυση κώδικα σε βάθος

**1. TF-IDF Search <br> Τι κάνει:**<br>
Δημιουργεί έναν πίνακα TF-IDF (Term Frequency-Inverse Document Frequency) που υπολογίζει τη σημασία κάθε λέξης σε κάθε παράγραφο.<br>
Υπολογίζει την ομοιότητα του ερωτήματος με τις παραγράφους χρησιμοποιώντας cosine similarity.<br>
**<br>Σημαντικές εντολές:**<br>
**1. compute_tfidf(paragraphs):<br>**
Δημιουργεί έναν TF-IDF πίνακα όπου κάθε γραμμή είναι μια παράγραφος και κάθε στήλη μια μοναδική λέξη.<br>
Η σημασία της λέξης υπολογίζεται με βάση τη συχνότητα εμφάνισης της λέξης στη συγκεκριμένη παράγραφο (TF) και πόσες παραγράφους περιέχουν τη λέξη (IDF).<br><br>
**2. search_tfidf(query, tfidf_matrix, feature_names):<br>**
Επεξεργάζεται το query για να το μετατρέψει σε TF-IDF μορφή.<br>
Υπολογίζει την ομοιότητα ανάμεσα στο query και στις παραγράφους με το cosine similarity.<br>
Τα αποτελέσματα κατατάσσονται με βάση την ομοιότητα <br><br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>

**2. Boolean Retrieval <br> Τι κάνει:<br>**
Χρησιμοποιεί ένα αντεστραμμένο ευρετήριο (inverted index) για να αναζητήσει παραγράφους βάσει των λογικών τελεστών AND, OR, NOT.<br>

**<br>Σημαντικές εντολές: <br> 1. preprocess_text(query):**<br>
Προετοιμάζει το query (π.χ., αφαίρεση stop words, lemmatization) για σύγκριση με το ευρετήριο.<br>
**1. create_inverted_index(paragraphs):<br>**
Δημιουργεί έναν χάρτη λέξεων -> λίστες παραγράφων όπου εμφανίζονται.<br>
**2. operator_stack:<br>**
Περιέχει τους λογικούς τελεστές (AND, OR, NOT) που βρίσκονται στο query.<br>
**3. results.update(InvertedList[word]):<br>**
Προσθέτει τις παραγράφους που περιέχουν τη λέξη στο τελικό αποτέλεσμα.<br>
**4. results.intersection:<br>**
Επιστρέφει τις κοινές παραγράφους (AND).<br>
**5. set(range(len(paragraphs))) - excluded_paragraphs:<br>**
Εξαιρεί παραγράφους που περιέχουν μια συγκεκριμένη λέξη (NOT).<br> <br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>

**3. Okapi BM25<br> Τι κάνει:<br>**
Υπολογίζει τη σχετικότητα μιας παραγράφου με το query, βασισμένη στο BM25.<br>
Το BM25 δίνει μεγαλύτερη σημασία σε λέξεις που εμφανίζονται μεσαία συχνά (όχι υπερβολικά συχνά ούτε σπάνια).<br>
**<br>Σημαντικές εντολές:<br>**
compute_bm25(paragraphs):<br>
Δημιουργεί ένα μοντέλο BM25, λαμβάνοντας υπόψη το μήκος των παραγράφων και τις συχνότητες των λέξεων.<br>
**1. search_bm25(query, bm25 , paragraphs):<br>**
Αναλύει το query.<br>
Υπολογίζει τη βαθμολογία σχετικότητας για κάθε παράγραφο.<br>
Επιστρέφει τα πιο σχετικά αποτελέσματα.<br><br>
**4. Επεξεργασία Ερωτήματος<br> Τι κάνει:<br>**
Επεξεργάζεται το query πριν χρησιμοποιηθεί από τον αλγόριθμο αναζήτησης.<br>
Αφαιρεί stop words, μετατρέπει σε μικρά γράμματα, και εκτελεί lemmatization.<br>
**<br> Σημαντικές εντολές:<br> 1. preprocess_text(query):<br>**
Κάνει όλες τις απαραίτητες επεξεργασίες (π.χ., αφαίρεση ειδικών χαρακτήρων, stop words).<br>
**2. original_query = query.split():<br>**
Χωρίζει το query σε λίστα λέξεων.<br>
**3. preprocess_text(" ".join(words)):<br>**
Επεξεργάζεται το query αφού αφαιρεθούν οι τελεστές (AND, OR, NOT).<br><br>
____________________________________________________________________________________________________________________________________________________________________________________________________________________________________
<br>

**5. Δομή Μενού Επιλογών<br> Τι κάνει:<br>**
Ζητά από τον χρήστη να επιλέξει μέθοδο αναζήτησης και εκτελεί τη συγκεκριμένη μέθοδο.<br>
**<br>Σημαντικές εντολές:<br> 1. input("Enter choice (1/2/3/4): "):<br>**
Ζητά από τον χρήστη να εισάγει την επιλογή του.<br>
**2. if choice == '4':<br>**
Ελέγχει αν ο χρήστης θέλει να τερματίσει το πρόγραμμα.<br>
**3. elif choice == '1':<br>**
Ελέγχει αν ο χρήστης επέλεξε TF-IDF και εκτελεί τον αντίστοιχο αλγόριθμο.<br>


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity
# α) Βήμα 4. Μηχανή αναζήτησης Επεξεργασία ερωτήματος (Query Processing): 
# β) Βήμα 4. Μηχανή αναζήτησης (Search Engine): β) Κατάταξη αποτελεσμάτων (Ranking)
def search_query(paragraphs):
    print("Choose search algorithm:")
    print("1. TF-IDF")
    print("2. Boolean Retrieval")
    print("3. Okapi BM25")
    print("4. Exit")
    
    # Επιλογή της μεθόδου αναζήτησης πρώτα
    choice = input("Enter choice (1/2/3/4): ")

    if choice == '4':
        print("Exiting search.")
        return  # Επιστροφή για να σταματήσει η λειτουργία

    # Εισαγωγή του query μετά την επιλογή της μεθόδου
    query = input("Enter your search query: ")

    if choice == '1':
        # TF-IDF
        print("\n--- Running TF-IDF Search ---")
        tfidf_matrix, feature_names = compute_tfidf(paragraphs)
        similarities = search_tfidf(query, tfidf_matrix, feature_names)
        
        # Εκτύπωση των αποτελεσμάτων βάσει της ομοιότητας
        ranked_paragraphs = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)
        print("TF-IDF Search Results:")
        for idx, score in ranked_paragraphs[:5]:  # Εμφάνιση των 5 πιο σχετικών παραγράφων
            print(f"Score: {score:.4f} - {paragraphs[idx]}")

    elif choice == '2':
        # Boolean Retrieval
        print("\n--- Running Boolean Retrieval Search ---")
        original_query = query.split()  # Αποθήκευση του αρχικού κειμένου για επεξεργασία
        SearchingList = preprocess_text(query)  # Προεπεξεργασία του ερωτήματος
        
        print(f"Processed search query: {', '.join(SearchingList)}")
        InvertedList = create_inverted_index(paragraphs)

        # Χωρίζουμε το query σε λέξεις και τελεστές (AND, OR, NOT)
        operators = {'AND', 'OR', 'NOT'}
        words = []
        operator_stack = []

        # Χωρίζουμε τις λέξεις από τους τελεστές
        for word in original_query:
            if word.upper() in operators:
                operator_stack.append(word.upper())
            else:
                words.append(word)

        SearchingList = preprocess_text(" ".join(words))

        array = []
        for word in SearchingList:
            if word in InvertedList:
                array.append(word)

        # Διαχείριση της αναζήτησης με AND, OR ή χωρίς τελεστές
        results = set()
        if 'AND' in operator_stack:
            if array:
                results = set(InvertedList[array[0]])  # Ξεκινάμε με τα αποτελέσματα του πρώτου όρου
                for word in array[1:]:
                    results = results.intersection(set(InvertedList[word]))
        elif 'OR' in operator_stack:
            for word in array:
                if word in InvertedList:
                    results.update(InvertedList[word])       
        elif 'NOT' in operator_stack:
            # Διαχείριση NOT: Βρίσκουμε παραγράφους που δεν περιέχουν τη λέξη
                try:
                    word_to_exclude_index = operator_stack.index('NOT')
                    if word_to_exclude_index + 1 < len(words):
                        word_to_exclude = preprocess_text(words[word_to_exclude_index + 1])[0]
                        excluded_paragraphs = set()
                        if word_to_exclude in InvertedList:
                           # Αφαιρούμε τις παραγράφους που περιέχουν τη λέξη από τα αποτελέσματα
                            excluded_paragraphs = set(InvertedList[word_to_exclude])
                            #exclusion_results = set(InvertedList[word_to_exclude])
                        if array:
                            results = set(InvertedList[array[0]])
                            for word in array[1:]:
                                results = results.intersection(set(InvertedList[word]))
                        else:
                            results = set(range(len(paragraphs)))

                        results = results - excluded_paragraphs  # Αφαιρούμε τις παραγράφους που περιέχουν τη λέξη
                        print(f"Excluded paragraphs: {excluded_paragraphs}")
                    else:
                        print("'NOT' must be followed by a word, WRONG FORMAT")
                except ValueError:
                    print("Invalid query format: 'NOT' must be followed by a word.")
        else:
            #Χωρίς τελεστή: αναζήτηση για όλες τις λέξεις
            for word in array:
                if word in InvertedList:
                    results.update(InvertedList[word])  # Χρησιμοποιούμε την μέθοδο "update"

        # Εκτύπωση αποτελεσμάτων
        if results:
            print("Matching paragraphs:")
            for idx in sorted(results):
                print(idx)
        elif excluded_paragraphs:
            for idx in sorted(excluded_paragraphs):
                print(idx)
        else:
            print("No matches found.")

    elif choice == '3':
        # Okapi BM25
        print("\n--- Running Okapi BM25 Search ---")
        bm25 = compute_bm25(paragraphs)
        search_bm25(query, bm25, paragraphs)


    else:
        print("Invalid choice! Please try again.")

search_(all_paragraphs)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
from sklearn.metrics.pairwise import cosine_similarity



nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()


# Βήμα 4. Μηχανή αναζήτησης (Search Engine): β) Κατάταξη αποτελεσμάτων (Ranking)
def compute_tfidf(paragraphs):
    # Δημιουργία του TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Εφαρμογή του vectorizer για να υπολογίσουμε τα TF-IDF scores για τις παραγράφους
    tfidf_matrix = tfidf_vectorizer.fit_transform(paragraphs)
    
    # Επιστροφή του TF-IDF matrix και των χαρακτηριστικών (feature names)
    return tfidf_matrix, tfidf_vectorizer

def search_tfidf(query, tfidf_matrix, tfidf_vectorizer):
    # Μετατροπή του ερωτήματος σε TF-IDF vector
    query_vector = tfidf_vectorizer.transform([query])  # Μετατροπή του query σε vector
    
    # Υπολογισμός της ομοιότητας του query με κάθε παράγραφο
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    return similarities


def compute_bm25(paragraphs):
    # Προεπεξεργασία και μετατροπή των παραγράφων σε λίστες λέξεων
    tokenized_paragraphs = [preprocess_text(paragraph) for paragraph in paragraphs]
    
    # Δημιουργία του μοντέλου BM25
    bm25 = BM25Okapi(tokenized_paragraphs)
    
    return bm25


def search_bm25(query, bm25, paragraphs):
    # Προεπεξεργασία του ερωτήματος
    query_tokens = preprocess_text(query)
    
    # Υπολογισμός των αποτελεσμάτων BM25
    scores = bm25.get_scores(query_tokens)
    
    # Εκτύπωση των αποτελεσμάτων με ταξινόμηση
    ranked_paragraphs = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    
    print("BM25 Search Results:")
    
    # Περιορισμός των αποτελεσμάτων σε 5 αν υπάρχουν περισσότερα
    num_results = min(5, len(ranked_paragraphs))
    
    # Έλεγχος για το αν οι δείκτες είναι έγκυροι
    for idx, score in ranked_paragraphs[:num_results]:
        if idx < len(paragraphs):  # Αν ο δείκτης είναι εντός των ορίων
            print(f"Score: {score:.4f} - {paragraphs[idx]}")
        else:
            print(f"Index {idx} out of range for paragraphs.")


# Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):
def preprocess_text(text):
    # Το αποτέλεσμα του βήματος περνάει στο επόμενο
    # 1. Αφαίρεση ειδικών χαρακτήρων και αριθμών
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 2. Tokenization: Διαχωρισμός κειμένου σε λέξεις
    tokens = word_tokenize(text.lower())  # Όλα τα γράμματα σε πεζά
    
    # 3. Αφαίρεση stop words
    tokens = [word for word in tokens if word not in stop_words]
    
    # 4. Lemmatization: Μείωση σε βασική μορφή λέξης
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    #Επιστρέφω την τελική λίστα
    return lemmatized_tokens



# Βήμα 3. Ευρετήριο (Indexing):
def create_inverted_index(paragraphs):
    #δημιουργία άδειου λεξικού
    inverted_index = {}
    
    for idx, paragraph in enumerate(paragraphs):
        
        # Προεπεξεργασία κάθε παραγράφου
        lemmatized_tokens = preprocess_text(paragraph)
        
        for word in lemmatized_tokens:
            if word not in inverted_index:
                inverted_index[word] = []
            inverted_index[word].append(idx)  # Προσθήκη του index της παραγράφου στη λέξη
    
    #Επιστρέφω τo τελικό λεξικό , ευρετήριο
    return inverted_index


# α) Βήμα 4. Μηχανή αναζήτησης Επεξεργασία ερωτήματος (Query Processing): 
# β) Βήμα 4. Μηχανή αναζήτησης (Search Engine): β) Κατάταξη αποτελεσμάτων (Ranking)
def search_query(paragraphs):
    print("Choose search algorithm:")
    print("1. TF-IDF")
    print("2. Boolean Retrieval")
    print("3. Okapi BM25")
    print("4. Exit")
    
    # Επιλογή της μεθόδου αναζήτησης πρώτα
    choice = input("Enter choice (1/2/3/4): ")

    if choice == '4':
        print("Exiting search.")
        return  # Επιστροφή για να σταματήσει η λειτουργία

    # Εισαγωγή του query μετά την επιλογή της μεθόδου
    query = input("Enter your search query: ")

    if choice == '1':
        # TF-IDF
        print("\n--- Running TF-IDF Search ---")
        tfidf_matrix, feature_names = compute_tfidf(paragraphs)
        similarities = search_tfidf(query, tfidf_matrix, feature_names)
        
        # Εκτύπωση των αποτελεσμάτων βάσει της ομοιότητας
        ranked_paragraphs = sorted(enumerate(similarities), key=lambda x: x[1], reverse=True)
        print("TF-IDF Search Results:")
        for idx, score in ranked_paragraphs[:5]:  # Εμφάνιση των 5 πιο σχετικών παραγράφων
            print(f"Score: {score:.4f} - {paragraphs[idx]}")

    elif choice == '2':
        # Boolean Retrieval
        print("\n--- Running Boolean Retrieval Search ---")
        original_query = query.split()  # Αποθήκευση του αρχικού κειμένου για επεξεργασία
        SearchingList = preprocess_text(query)  # Προεπεξεργασία του ερωτήματος
        
        print(f"Processed search query: {', '.join(SearchingList)}")
        InvertedList = create_inverted_index(paragraphs)

        # Χωρίζουμε το query σε λέξεις και τελεστές (AND, OR, NOT)
        operators = {'AND', 'OR', 'NOT'}
        words = []
        operator_stack = []

        # Χωρίζουμε τις λέξεις από τους τελεστές
        for word in original_query:
            if word.upper() in operators:
                operator_stack.append(word.upper())
            else:
                words.append(word)

        SearchingList = preprocess_text(" ".join(words))

        array = []
        for word in SearchingList:
            if word in InvertedList:
                array.append(word)

        # Διαχείριση της αναζήτησης με AND, OR ή χωρίς τελεστές
        results = set()
        if 'AND' in operator_stack:
            if array:
                results = set(InvertedList[array[0]])  # Ξεκινάμε με τα αποτελέσματα του πρώτου όρου
                for word in array[1:]:
                    results = results.intersection(set(InvertedList[word]))
        elif 'OR' in operator_stack:
            for word in array:
                if word in InvertedList:
                    results.update(InvertedList[word])       
        elif 'NOT' in operator_stack:
            # Διαχείριση NOT: Βρίσκουμε παραγράφους που δεν περιέχουν τη λέξη
                try:
                    word_to_exclude_index = operator_stack.index('NOT')
                    if word_to_exclude_index + 1 < len(words):
                        word_to_exclude = preprocess_text(words[word_to_exclude_index + 1])[0]
                        excluded_paragraphs = set()
                        if word_to_exclude in InvertedList:
                           # Αφαιρούμε τις παραγράφους που περιέχουν τη λέξη από τα αποτελέσματα
                            excluded_paragraphs = set(InvertedList[word_to_exclude])
                            #exclusion_results = set(InvertedList[word_to_exclude])
                        if array:
                            results = set(InvertedList[array[0]])
                            for word in array[1:]:
                                results = results.intersection(set(InvertedList[word]))
                        else:
                            results = set(range(len(paragraphs)))

                        results = results - excluded_paragraphs  # Αφαιρούμε τις παραγράφους που περιέχουν τη λέξη
                        print(f"Excluded paragraphs: {excluded_paragraphs}")
                    else:
                        print("'NOT' must be followed by a word, WRONG FORMAT")
                except ValueError:
                    print("Invalid query format: 'NOT' must be followed by a word.")
        else:
            #Χωρίς τελεστή: αναζήτηση για όλες τις λέξεις
            for word in array:
                if word in InvertedList:
                    results.update(InvertedList[word])  # Χρησιμοποιούμε την μέθοδο "update"
  
        # Εκτύπωση αποτελεσμάτων
        if results:
            print("Matching paragraphs:")
            for idx in sorted(results):
                print(idx)
        elif excluded_paragraphs:
            for idx in sorted(excluded_paragraphs):
                print(idx)
        else:
            print("No matches found.")

    elif choice == '3':
        # Okapi BM25
        print("\n--- Running Okapi BM25 Search ---")
        bm25 = compute_bm25(paragraphs)
        search_bm25(query, bm25, paragraphs)


    else:
        print("Invalid choice! Please try again.")






for paragraph in all_paragraphs:
    # Εκτύπωση του αρχικού κειμένου
    print("Πριν την προ-επεξεργασία:")
    print(paragraph)
    print("\n" + "-"*50 + "\n")
                
    # Εφαρμογή προ-επεξεργασίας στην παράγραφο
    cleaned_paragraph = preprocess_text(paragraph)
    

inverted_index = create_inverted_index(all_paragraphs)
print("Αντεστραμμένο Ευρετήριο:")
for word, paragraphs in inverted_index.items():
    print(f"Λέξη: {word} | παράγραφοι : {', '.join(map(str, paragraphs))}")


search_query(all_paragraphs)

Choose search algorithm:
1. TF-IDF
2. Boolean Retrieval
3. Okapi BM25
4. Exit
Exiting search.



# Γενικά Συμπεράσματα:
Για αρχή επιλέξαμε τη μέθοδο TF-IDF και την λέξη "pain"<br>

**Καταλληλότητα Αποτελεσμάτων:**<br>
Τα αποτελέσματα ανταποκρίνονται καλά στο ερώτημα "πόνος", εστιάζοντας στη θεραπεία διαφορετικών τύπων πόνου, όπως ο χρόνιος, οξύς, καρκινικός, μετεγχειρητικός, ημικρανία.<br>

**Κατανομή Σκορ:**<br>
Τα υψηλότερα σκορ προκύπτουν από κείμενα που περιλαμβάνουν πολλές αναφορές στη λέξη "pain" σε διάφορα πλαίσια, όπως χρόνιος πόνος, μετεγχειρητικός πόνος κ.λπ.<br>

**Ελλείψεις:**<br>
Το μοντέλο TF-IDF δεν κατανοεί το νόημα των συμφραζόμενων και βασίζεται μόνο στη συχνότητα της λέξης "pain". Για παράδειγμα, το 5ο αποτέλεσμα (που αφορά οδοντιατρικό πόνο) είναι σχετικό, αλλά έχει χαμηλότερο σκορ.<br>

**Εφαρμογές:**<br>
Οι πληροφορίες μπορούν να χρησιμοποιηθούν για τη σύγκριση αναλγητικών ανάλογα με τον τύπο του πόνου ή για περαιτέρω ανάλυση της αποτελεσματικότητάς τους.<br>

Choose search algorithm:
1. TF-IDF
2. Boolean Retrieval
3. Okapi BM25
4. Exit

--- Running TF-IDF Search ---
TF-IDF Search Results:
Score: 0.5149 - A transdermal patch is available for the treatment of chronic pain.[18] These patches are not indicated for use in acute pain, pain that is expected to last only for a short period, or pain after surgery, nor are they recommended for opioid addiction.[41]
Score: 0.4664 - Paracetamol is used for the relief of mild to moderate pain such as headache, muscle aches, minor arthritis pain, toothache as well as pain caused by cold, flu, sprains, and dysmenorrhea.[61] It is recommended, in particular, for acute mild to moderate pain, since the evidence for the treatment of chronic pain is insufficient.[15]
Score: 0.4150 - The studies to support or refute the use of paracetamol for cancer pain and neuropathic pain are lacking.[31][32] There is limited evidence in favor of the use of the intravenous form of paracetamol for acute pain control in the emergency department.[74] The combination of paracetamol with caffeine is superior to paracetamol alone for the treatment of acute pain.[75]
Score: 0.3732 - Paracetamol relieves pain in both acute mild migraine and episodic tension headache.[16][17] At a standard dose, paracetamol slightly reduces fever;[14][18][19] it is inferior to ibuprofen in that respect,[20] and the benefits of its use for fever are unclear, particularly in the context of fever of viral origins.[14][21][22]  The aspirin/paracetamol/caffeine combination also helps with both conditions where the pain is mild and is recommended as a first-line treatment for them.[23][24] Paracetamol is effective for post-surgical pain, but it is inferior to ibuprofen.[25] The paracetamol/ibuprofen combination provides further increase in potency and is superior to either drug alone.[25][26] The pain relief paracetamol provides in osteoarthritis is small and clinically insignificant.[15][27][28] The evidence in its favor for the use in low back pain, cancer pain, and neuropathic pain is insufficient.[15][27][29][30][31][32]
Score: 0.2848 - Pain after a dental surgery provides a reliable model for the action of analgesics on other kinds of acute pain.[66] For the relief of such pain, paracetamol is inferior to ibuprofen.[25] Full therapeutic doses of nonsteroidal anti-inflammatory drugs (NSAIDs) ibuprofen, naproxen or diclofenac are clearly more efficacious than the paracetamol/codeine combination which is frequently prescribed for dental pain.[67] The combinations of paracetamol and NSAIDs ibuprofen or diclofenac are promising, possibly offering better pain control than either paracetamol or the NSAID alone.[25][26][68][69] Additionally, the paracetamol/ibuprofen combination may be superior to paracetamol/codeine and ibuprofen/codeine combinations.[26]

# Γενικά Συμπεράσματα:
Για αρχή επιλέξαμε τη μέθοδο Οkapi25 και την λέξη "pain"<br>

**Καταλληλότητα Αποτελεσμάτων:<br>**
Τα αποτελέσματα ανταποκρίνονται καλά στο ερώτημα "πόνος", εστιάζοντας στη θεραπεία διαφορετικών τύπων πόνου, όπως ο χρόνιος, οξύς, καρκινικός, μετεγχειρητικός, ημικρανία.<br>

**Κατανομή Σκορ:<br>**
Τα υψηλότερα σκορ προκύπτουν από κείμενα που περιλαμβάνουν πολλές αναφορές στον "πόνο" σε διάφορα πλαίσια, όπως χρόνιος πόνος, μετεγχειρητικός πόνος κ.λπ.<br>

**Ελλείψεις:<br>**
Το μοντέλο TF-IDF δεν κατανοεί το νόημα των συμφραζόμενων και βασίζεται μόνο στη συχνότητα της λέξης "πόνος". Για παράδειγμα, το 5ο αποτέλεσμα (που αφορά οδοντιατρικό πόνο) είναι σχετικό, αλλά έχει χαμηλότερο σκορ.<br>

**Εφαρμογές:<br>**
Οι πληροφορίες μπορούν να χρησιμοποιηθούν για τη σύγκριση αναλγητικών ανάλογα με τον τύπο του πόνου ή για περαιτέρω ανάλυση της αποτελεσματικότητάς τους.<br>

Choose search algorithm:
1. TF-IDF
2. Boolean Retrieval
3. Okapi BM25
4. Exit

--- Running Okapi BM25 Search ---
BM25 Search Results:
Score: 2.4870 - A transdermal patch is available for the treatment of chronic pain.[18] These patches are not indicated for use in acute pain, pain that is expected to last only for a short period, or pain after surgery, nor are they recommended for opioid addiction.[41]
Score: 2.4608 - Paracetamol is used for the relief of mild to moderate pain such as headache, muscle aches, minor arthritis pain, toothache as well as pain caused by cold, flu, sprains, and dysmenorrhea.[61] It is recommended, in particular, for acute mild to moderate pain, since the evidence for the treatment of chronic pain is insufficient.[15]
Score: 2.3552 - The studies to support or refute the use of paracetamol for cancer pain and neuropathic pain are lacking.[31][32] There is limited evidence in favor of the use of the intravenous form of paracetamol for acute pain control in the emergency department.[74] The combination of paracetamol with caffeine is superior to paracetamol alone for the treatment of acute pain.[75]
Score: 2.3089 - Paracetamol relieves pain in both acute mild migraine and episodic tension headache.[16][17] At a standard dose, paracetamol slightly reduces fever;[14][18][19] it is inferior to ibuprofen in that respect,[20] and the benefits of its use for fever are unclear, particularly in the context of fever of viral origins.[14][21][22]  The aspirin/paracetamol/caffeine combination also helps with both conditions where the pain is mild and is recommended as a first-line treatment for them.[23][24] Paracetamol is effective for post-surgical pain, but it is inferior to ibuprofen.[25] The paracetamol/ibuprofen combination provides further increase in potency and is superior to either drug alone.[25][26] The pain relief paracetamol provides in osteoarthritis is small and clinically insignificant.[15][27][28] The evidence in its favor for the use in low back pain, cancer pain, and neuropathic pain is insufficient.[15][27][29][30][31][32]
Score: 2.2155 - Pain after a dental surgery provides a reliable model for the action of analgesics on other kinds of acute pain.[66] For the relief of such pain, paracetamol is inferior to ibuprofen.[25] Full therapeutic doses of nonsteroidal anti-inflammatory drugs (NSAIDs) ibuprofen, naproxen or diclofenac are clearly more efficacious than the paracetamol/codeine combination which is frequently prescribed for dental pain.[67] The combinations of paracetamol and NSAIDs ibuprofen or diclofenac are promising, possibly offering better pain control than either paracetamol or the NSAID alone.[25][26][68][69] Additionally, the paracetamol/ibuprofen combination may be superior to paracetamol/codeine and ibuprofen/codeine combinations.[26]

# Boolean Retrival : OR
Η μέθοδος Boolean Retrieval στην επιλογή OR επιστρέφει τα κείμενα που περιέχουν τουλάχιστον μία από τις δύο λέξεις. <br>
Στην προκειμένη περίπτωση, τα αποτελέσματα περιέχουν είτε τη λέξη "gi" είτε τη λέξη "vet".

OR
Choose search algorithm:
1. TF-IDF
2. Boolean Retrieval
3. Okapi BM25
4. Exit

--- Running Boolean Retrieval Search ---
Processed search query: gi, vet
Matching paragraphs:
109
110

# Boolean Retrival : OR
Η μέθοδος Boolean Retrieval στην επιλογή AND επιστρέφει τα κείμενα που περιέχουν και τις δύο λέξεις ταυτόχρονα. 
<br> Στην προκειμένη περίπτωση, τα κείμενα που επιστρέφονται περιέχουν τόσο τη λέξη "snake" όσο και τη λέξη "brown".

AND
Choose search algorithm:
1. TF-IDF
2. Boolean Retrieval
3. Okapi BM25
4. Exit

--- Running Boolean Retrieval Search ---
Processed search query: snake, brown
Matching paragraphs:
111

# Boolean Retrival : NOT

Η μέθοδος Boolean Retrieval στην επιλογή NOT επιστρέφει τα κείμενα που περιέχουν τη λέξη που προηγείται του "NOT", ενώ εξαιρεί αυτά που περιλαμβάνουν τη λέξη που ακολουθεί.<br>
Στην προκειμένη περίπτωση, τα κείμενα που επιστρέφονται περιέχουν τη λέξη "sometimes" αλλά όχι τη λέξη "referred".<br>
Λόγω της δυσκολίας σωστής εκτέλεσης της συναρτησης, εμφανίζονται τα κείμενα που απορρίπτονται.<br>

NOT
Choose search algorithm:
1. TF-IDF
2. Boolean Retrieval
3. Okapi BM25
4. Exit

--- Running Boolean Retrieval Search ---
Processed search query: sometimes, referred
Excluded paragraphs: {105}
105


# ΣΗΜΕΙΩΣΗ
Ενσωματώσαμε το πρόγραμμα στο τέλος και προχωρήσαμε στη διαδικασία copy-paste των εξόδων από τις επιλογές του μενού χρήστη, καθώς αντιμετωπίσαμε δυσκολίες στην κατανόηση του τρόπου εκτέλεσης των λειτουργιών ξεχωριστά.